### See what random forest does with best features so far.

This file shows what the default random forest does with best feature engineering so far.

Fits in 6 minutes.
Aggregate log loss on test set: 0.1707
Average accuracy score for all targets : 0.977
Average F1 score for all targets : 0.976

So it's pretty good, but not quite as good as best logistic regression.  Some tuning may help.

In [1]:
#### Imports/setup

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', 60)

from timeit import default_timer as timer

# for the pipeline
from sklearn.pipeline import Pipeline
# for the selectors
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MaxAbsScaler, Imputer
# for gluing preprocessed text and numbers together
from sklearn.pipeline import FeatureUnion

# Import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

# Import CountVectorizer and HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer

from sklearn.feature_selection import chi2, SelectKBest

# metrics
from sklearn.metrics import f1_score, accuracy_score, classification_report

# unflattener
import python.flat_to_labels as ftl

#### Set up a train-test split making sure we have all labels in both splits
from python.multilabel import multilabel_train_test_split

from python.dd_mmll import multi_multi_log_loss, BOX_PLOTS_COLUMN_INDICES

C:\Users\saus\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


#### Load the data

In [2]:
# Get data
the_data = pd.read_csv('../data/TrainingData.csv', index_col=0)

# take a look
the_data.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,NaN,NaN,Teacher-Elementary,NaN,NaN,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,CONTRACTOR SERVICES,BOND EXPENDITURES,BUILDING FUND,(blank),Regular,NaN,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,NaN,NaN,TCHER 2ND GRADE,NaN,Regular Instruction,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,EMPLOYEE BENEFITS,TEACHER SUBS,GENERAL FUND,"Teacher, Short Term Sub",Regular,NaN,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,TEACHER COVERAGE FOR TEACHER,TEACHER SUBS,GENERAL FUND,"Teacher, Secondary (High)",Alternative,NaN,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


####  Encode the targets as categorical variables

In [3]:
### bind variable LABELS - these are actually the targets and we're going to one-hot encode them...
LABELS = ['Function',  'Use',  'Sharing',  'Reporting',  'Student_Type',  'Position_Type', 
          'Object_Type',  'Pre_K',  'Operating_Status']

### This turns out to be key.  Submission requires the dummy versions of these vars to be in this order.
LABELS.sort()

# Define the lambda function: categorize_label
categorize_label = lambda x: x.astype('category')

# Convert df[LABELS] to a categorical type
the_data[LABELS] = the_data[LABELS].apply(categorize_label, axis=0)

# Print the converted dtypes
print(the_data[LABELS].dtypes)

Function            category
Object_Type         category
Operating_Status    category
Position_Type       category
Pre_K               category
Reporting           category
Sharing             category
Student_Type        category
Use                 category
dtype: object


#### Save the unique labels for each output (category)

In [4]:
# build a dictionary
the_labels = {col : the_data[col].unique().tolist() for col in the_data[LABELS].columns}
# take a look at one entry
the_labels['Use']

['Instruction',
 'NO_LABEL',
 'O&M',
 'Pupil Services & Enrichment',
 'ISPD',
 'Leadership',
 'Business Services',
 'Untracked Budget Set-Aside']

In [5]:
[(col, len(the_labels[col])) for col in the_data[LABELS].columns]

[('Function', 37),
 ('Object_Type', 11),
 ('Operating_Status', 3),
 ('Position_Type', 25),
 ('Pre_K', 3),
 ('Reporting', 3),
 ('Sharing', 5),
 ('Student_Type', 9),
 ('Use', 8)]

#### Change fraction to suit.
Note: small fractions will have a hard time ensuring labels in both splits.

In [6]:
# downsize it or not
df = the_data.sample(frac=0.15, random_state=777)
# df = the_data

#### Get targets as set of one-hot encoded columns

In [7]:
# name these columns
NUMERIC_COLUMNS = ['FTE', 'Total']

# Get labels and convert to dummy variables: label_dummies
label_dummies = pd.get_dummies(df[LABELS])

In [8]:
label_dummies.shape

(60042, 104)

#### Setting up a train-test split  for modeling

#### =========== Begin ModRF; use random forest on features from best logistic regression model  ======================

Some things to note about the default CountVectorizer:
1) All strings are downcased
2) The default setting selects tokens of 2 or more alphanumeric characters (punctuation is completely ignored and always treated as a token separator).  This means single letter or digit tokens are ignored.
3) If the vectorizer is used to transform another input (e.g. test), any tokens not in the original corpus are ignored.

In [9]:
# define combine_text_columns()
def combine_text_columns(df, to_drop=NUMERIC_COLUMNS + LABELS):
    """ converts all text columns in each row of df to single string """
    # Drop non-text columns that are in the df
    to_drop = set(to_drop) & set(df.columns.tolist())
    text_data = df.drop(to_drop, axis=1)  
    # Replace nans with blanks
    text_data.fillna('', inplace=True)    
    # Join all text items in a row that have a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

#### For RF, it's okay to use the labels directly (instead of binarizing).  The probabilities will come out the way we need them.  They may be in a list with one element per target.

Slight change of plans: multilabel_train_test_split only works with binary indicator matrices.  So a quick workaround is to dummy the labels, then do the split.  Then use the indices of the y_train/y_test to get the original ys that we want to use.

For mmll we need the label probabilities as array of shape (num_samples, 104).  Can get this format by calling np.hstack on proba output.

Another ramification: mmll also wants ys in binarized format.  So we need those to do the comparison.  We have them from the dummy variable version of the original input.

In [10]:
# Import FunctionTransformer
from sklearn.preprocessing import FunctionTransformer

# Get the dummy encoding of the labels
dummy_labels = pd.get_dummies(df[LABELS])

# Get the features in the data
NON_LABELS = [c for c in df.columns if c not in LABELS]

# Split into training and test sets
X_train, X_test, y_train, y_test = multilabel_train_test_split(df[NON_LABELS],
                                                               dummy_labels,
                                                               0.2, 
                                                               seed=123)
# Preprocess the text data: get_text_data
get_text_data = FunctionTransformer(combine_text_columns, validate=False)

# Use all 0s instead of noise: get_numeric_data
get_numeric_data_hack = FunctionTransformer(lambda x: np.zeros(x[NUMERIC_COLUMNS].shape, dtype=np.float), validate=False)

#### Swapping the binarized labels for the real labels.

In [11]:
y_train_bin = y_train; y_test_bin = y_test

In [12]:
### now get the original ys
y_train = df.loc[y_train_bin.index, LABELS]

In [13]:
y_test = df.loc[y_test_bin.index, LABELS]; y_test.shape

(12008, 9)

#### Check the shapes.

In [14]:
y_train.shape, y_test.shape

((48034, 9), (12008, 9))

In [15]:
y_train_bin.shape, y_test_bin.shape

((48034, 104), (12008, 104))

#### Check that all the columns have some true values (we need all the labels populated).

In [16]:
np.sort(y_train_bin.iloc[:,0:37].sum(axis=0))

array([    3,     4,     6,    16,    26,    37,    41,    62,    76,
          84,   154,   176,   210,   221,   231,   264,   335,   338,
         345,   391,   497,   501,   590,   704,   885,  1024,  1270,
        1528,  1716,  1773,  2296,  2336,  2386,  2416,  7139,  7501,
       10452], dtype=int64)

In [17]:
y_train['Function'].value_counts()

Teacher Compensation                               10452
Substitute Compensation                             7501
NO_LABEL                                            7139
Instructional Materials & Supplies                  2416
Facilities & Maintenance                            2386
Professional Development                            2336
Aides Compensation                                  2296
Student Transportation                              1773
Food Services                                       1716
School Administration                               1528
Enrichment                                          1270
Extended Time & Tutoring                            1024
Curriculum Development                               885
Physical Health & Services                           704
Social & Emotional                                   590
Library & Media                                      501
Special Population Program Management & Support      497
Data Processing & Information S

#### Make a classifier

In [18]:
# Insert random forest clf instead of onevsrest
ModRF = Pipeline([
    ('union', FeatureUnion(transformer_list = [
        ('numeric_features', Pipeline([
            ('selector', get_numeric_data_hack),
            ('imputer', Imputer())
        ])),
        ('text_features', Pipeline([
            ('selector', get_text_data),
            ('vectorizer', CountVectorizer(ngram_range=(1,2)))
        ]))
    ])),
    ('rf', RandomForestClassifier(n_jobs=-1))])

##### Fit  took 6 min with n_jobs=-1 and all default parameters

In [19]:
start = timer()
# Fit to the training data
ModRF.fit(X_train, y_train)
end = timer()
print('fit time: {:0.2f} seconds'.format(end - start))

fit time: 20.80 seconds


#### Get the probability predictions.

In [20]:
# get probas
start = timer()
ModRF_train_probas = ModRF.predict_proba(X_train)
ModRF_test_probas = ModRF.predict_proba(X_test)
end = timer()
print('Predict.proba time: {:0.2f} seconds'.format(end - start))

Predict.proba time: 4.39 seconds


#### Make sure the shapes are right.

In [21]:
np.hstack(ModRF_train_probas).shape, dummy_labels.loc[y_train.index, :].values.shape

((48034, 104), (48034, 104))

In [22]:
print('log loss on training set: {:0.4f}'.format(
    multi_multi_log_loss(np.hstack(ModRF_train_probas),
                         dummy_labels.loc[y_train.index, :].values, 
                         BOX_PLOTS_COLUMN_INDICES)))
print('log loss on test set: {:0.4f}'.format(
    multi_multi_log_loss(np.hstack(ModRF_test_probas),
                         dummy_labels.loc[y_test.index, :].values, 
                         BOX_PLOTS_COLUMN_INDICES)))

log loss on training set: 0.0444
log loss on test set: 0.3739


#### For standard metrics we need the yhats and the ys.

In [23]:
def report_f1(true, pred):
    the_scores = []
    for target in range(len(LABELS)):
        the_score = f1_score(true[:, target], pred[:, target], average='weighted')
        print('F1 score for target {}: {:.3f}'.format(LABELS[target], the_score))
        the_scores.append(the_score)
    print('Average F1 score for all targets : {:.3f}'.format(np.mean(the_scores)))

def report_accuracy(true, pred):
    the_scores = []
    for target in range(len(LABELS)):
        the_score = accuracy_score(true[:, target], pred[:, target])
        print('Accuracy score for target {}: {:.3f}'.format(LABELS[target], the_score))
        the_scores.append(the_score)
    print('Average accuracy score for all targets : {:.3f}'.format(np.mean(the_scores)))

#### For log loss we need binarized ys and probability predictions.

In [24]:
# grab the columns for each target - that's in BPCI indexed by position in LABELS
# normalize so probabilities sum to one (unless sum is zero, then we clip)

BPCI = BOX_PLOTS_COLUMN_INDICES

def norm_probs(probs, indices=BPCI, targets = LABELS):
    ''' input:  array n_samples, 104 ; output: array n_samples, 104 
         normalized within target columns such that for each target, the sum of probabilities for each row is 1'''
    # make a copy; don't want to smash the input
    lprobs = np.copy(probs)
    for i, targ in enumerate(targets):
        lprobs[:, indices[i]] /=  np.clip(np.sum(lprobs[:, indices[i]], axis=1, keepdims=True), 1e-15, np.inf)
    return lprobs

from sklearn.metrics import log_loss
def report_log_loss(true, pred):
    ''' Takes true in binarized format.  Both args are shape (n_samples, 104)'''
    the_scores = []
    # note: BPCI[idx] is the slice that gets the right columns for each target
    # first normalize probabilities within targets 
    normed_probas = norm_probs(pred)
    the_scores = []
    for idx, target in enumerate(LABELS):
        the_score = log_loss(true[:, BPCI[idx]], pred[:, BPCI[idx]])
        print('log loss for target {}: {:.3f}'.format(target, the_score))
        the_scores.append(the_score)
    print('Average log_loss for all targets : {:.3f}'.format(np.mean(the_scores))) 

#### Check shapes

In [25]:
y_test.shape, ftl.flat_to_labels(np.hstack(ModRF_test_probas)).shape

((12008, 9), (12008, 9))

In [26]:
report_f1(y_test.values, ftl.flat_to_labels(np.hstack(ModRF_test_probas)))

F1 score for target Function: 0.929
F1 score for target Object_Type: 0.966
F1 score for target Operating_Status: 0.981
F1 score for target Position_Type: 0.956
F1 score for target Pre_K: 0.985
F1 score for target Reporting: 0.970
F1 score for target Sharing: 0.956
F1 score for target Student_Type: 0.962
F1 score for target Use: 0.946
Average F1 score for all targets : 0.961


In [27]:
report_accuracy(y_test.values, ftl.flat_to_labels(np.hstack(ModRF_test_probas)))

Accuracy score for target Function: 0.930
Accuracy score for target Object_Type: 0.966
Accuracy score for target Operating_Status: 0.981
Accuracy score for target Position_Type: 0.956
Accuracy score for target Pre_K: 0.985
Accuracy score for target Reporting: 0.970
Accuracy score for target Sharing: 0.956
Accuracy score for target Student_Type: 0.962
Accuracy score for target Use: 0.947
Average accuracy score for all targets : 0.962


#### Result confirms DrivenData metric above.

In [28]:
report_log_loss(y_test_bin.values, np.hstack(ModRF_test_probas))

log loss for target Function: 0.801
log loss for target Object_Type: 0.340
log loss for target Operating_Status: 0.183
log loss for target Position_Type: 0.438
log loss for target Pre_K: 0.140
log loss for target Reporting: 0.259
log loss for target Sharing: 0.425
log loss for target Student_Type: 0.321
log loss for target Use: 0.458
Average log_loss for all targets : 0.374


In [29]:
# # Load the holdout data: holdout
# ### Over here the file is TestData.csv
# holdout = pd.read_csv('../data/TestData.csv', index_col=0)

In [30]:
# # start = timer()
# # # Generate predictions: predictions
# ModRF_predictions = ModRF.predict_proba(holdout)
# end = timer()
# print('predict time: {} seconds'.format(end - start))

In [31]:
# pred_Mod_RF = pd.DataFrame(columns=pd.get_dummies(df[LABELS], prefix_sep='__').columns, 
#                              index=holdout.index,
#                              data=ModRF_predictions)

# pred_Mod_RF.to_csv('pred_Mod_RF.csv')

***